In [ ]:
'''
Merging payroll notes with player contracts
'''
import pandas as pd
import unicodedata
from rapidfuzz import process, fuzz

contracts = pd.read_csv('../data/player_contracts.csv')

try:
    notes_raw = pd.read_csv('../data/payroll_notes.csv', encoding='utf-8')
except UnicodeDecodeError:
    notes_raw = pd.read_csv('../data/payroll_notes.csv', encoding='latin1')

notes_raw.columns = [col.strip() for col in notes_raw.columns]
contracts.columns = [col.strip() for col in contracts.columns]

if 'Unnamed: 1' in contracts.columns:
    contracts = contracts[contracts['Unnamed: 1'].ne('Player')]

notes_expanded = []
current_player = None
for _, row in notes_raw.iterrows():
    player = row['Player'] if 'Player' in row and pd.notnull(row['Player']) and str(row['Player']).strip() else current_player
    current_player = player
    for note in str(row['Notes']).split('.'):
        note = note.strip().strip(',')
        if note:
            notes_expanded.append({'Player': player, 'Note': note})

notes_df = pd.DataFrame(notes_expanded)

def normalize_name(name):
    if pd.isnull(name): return ''
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('utf-8').lower().strip()

contracts['Player_norm'] = contracts['Unnamed: 1'].apply(normalize_name)
notes_df['Player_norm'] = notes_df['Player'].apply(normalize_name)

def match_player(note_name, contract_names):
    match, score, idx = process.extractOne(note_name, contract_names, scorer=fuzz.token_sort_ratio)
    return match if score >= 90 else None

notes_df['Player_match'] = notes_df['Player_norm'].apply(lambda x: match_player(x, contracts['Player_norm']))

notes_grouped = notes_df.dropna(subset=['Player_match']).groupby('Player_match')['Note'].apply(list).reset_index()
contracts = contracts.merge(notes_grouped, left_on='Player_norm', right_on='Player_match', how='left')

for col in ['Player_norm', 'Player_match']:
    if col in contracts.columns:
        contracts = contracts.drop(col, axis=1)

if contracts.columns.tolist().count('Note') > 1:
    note_cols = [col for col in contracts.columns if col == 'Note']
    contracts = contracts.drop(note_cols[:-1], axis=1)

contracts.to_csv('../data/player_contracts_with_notes.csv', index=False)
print('Merged CSV saved as ../data/player_contracts_with_notes.csv')

Merged CSV saved as ../data/player_contracts_with_notes.csv


In [ ]:
'''
Change column names to more meaningful ones
'''
import pandas as pd

contracts = pd.read_csv('../data/player_contracts_with_notes.csv')

rename_map = {'Unnamed: 0': 'id', 'Unnamed: 1': 'name', 'Unnamed: 2': 'team'}
contracts = contracts.rename(columns=rename_map)

contracts.to_csv('../data/player_contracts_with_notes.csv', index=False)
print('Updated columns and saved as ../data/player_contracts_with_notes.csv')

Updated columns and saved as ../data/player_contracts_with_notes.csv
